In [40]:
import os
import numpy as np
import pandas as pd
from ast import literal_eval

In [25]:
class ProjectDirectoryError(Exception):
    """Raise for path and directory related exceptions."""
    
class CSVSchemaError(Exception):
    """Raise for .csv structure related exceptions."""

In [35]:
class DataSetUp:
    
    '''
    This class is used to:
       - Perform checks on directory structure
       - Validate the .csv schemas for all PRE and POST files  
    '''
    
    def __init__(self):
        
        self.project_path = os.path.dirname(os.getcwd()) # get project path
        self.raw_data_path = os.path.join(self.project_path, 'raw_data') # where all csv's should be saved
        # validate agreed directory structure
        if not os.path.isdir(self.raw_data_path):
            raise ProjectDirectoryError("Ensure the \raw_data project directory exists on your machine.")
        
        # extract .csv information
        csvs = [f for f in os.listdir(self.raw_data_path) if f[-4:].lower()=='.csv'] # list csv names
        self.csvs_post = [f for f in csvs if 'POST' in f.upper()] # POST files
        self.csvs_pre = [f for f in csvs if 'PRE' in f.upper()] # PRE files
        del csvs
        
    
    def check_schemas(self):

        '''
        Checks whether schemas are common with the sets of PRE and POST .csv files.

                Attribute Returns:
                    - pre_exceptions (pd.DataFrame): summary of PRE file column inconsistencies
                    - post_exceptions (pd.DataFrame): summary of POST file column inconsistencies
        '''
    
        pre_check = False # boolean to capture the existence of PRE file exceptions
        post_check = False # boolean to capture the existence of POST file exceptions
        
        # loop through both file types
        for csvs_set in [(self.csvs_pre, 'PRE files'), (self.csvs_post, 'POST files')]:
            cols = [] # container for column headings in all files
            
            # iterate thorugh each file
            for f in csvs_set[0]:
                # try utf-8 encoding
                try:
                    df = pd.read_csv(os.path.join(self.raw_data_path, f), encoding='utf-8')
                # otherwise try latin-1, as there are some encoding discrepancies in inputs
                except UnicodeDecodeError:
                    df = pd.read_csv(os.path.join(self.raw_data_path, f), encoding='latin-1')
                #df = df.rename(columns={'Tx Ecoul\nSem.\nN':'Défilement\nSem.\nN'})
                headings = list(df.columns.values)
                
                cols.append(headings) # store headings per file
            del df, f
            
            # check whether headers align
            for i, col_set in enumerate(cols):
                if i==0:
                    col_set_1 = col_set # col_set_1 stores the prior file' schema
                    check_sum = [] # instantiate list to store comparison results
                else:
                    check_sum.append(col_set==col_set_1) # append comparison
                    col_set_1 = col_set # store schema for next iteration
            del col_set, col_set_1, i
            
            # summarize exceptions if any are identified
            if np.array(check_sum).sum() != len(check_sum):  
                schema_exceptions = {} # instantiate container for specific exceptions
                # iterate through each csv file's columns
                for csv_num in range(len(cols)):
                    num_cols = len(cols[csv_num]) # number of columns
                    # create empty space / buffer to avoid DataFrame constructor errors later
                    buffer = np.array(np.ones((1,500-num_cols)), dtype="object") + np.nan
                    # store column names per .csv
                    heads = np.array(cols[csv_num]).reshape((1,num_cols))
                    # store .csv file headings as an array
                    schema_exceptions[csvs_set[0][csv_num]] = np.concatenate([heads, buffer], axis=1).tolist()[0]
                # convert dictionary array to DataFrame and remove empty buffer rows
                schema_exceptions_df = pd.DataFrame(schema_exceptions).dropna(how='all')
                indexer = [] # instantiate indexer for exceptions
                # check column-by-column for deviations across files
                for index in range(len(schema_exceptions_df)):
                    # check the number of unique values per column index
                    index_uniques = schema_exceptions_df.iloc[index,].T.drop_duplicates().shape[0]
                    # if the column name is not distinct; store the index position
                    if index_uniques != 1:
                        indexer.append(index)
                # overwrite the exceptions DataFrame using the exceptions indexer
                schema_exceptions_df = schema_exceptions_df.iloc[indexer]
                
                # create exception DataFrames per file type
                # PRE files
                if csvs_set[1] == 'PRE files':
                    pre_check = True # highlight where PRE exceptions are found
                    # write the DataFrame as an attribute
                    self.pre_exceptions = schema_exceptions_df.copy()
                    del schema_exceptions_df
                # POST files
                else:
                    post_check = True  # highlight where POST exceptions are found
                    # write the DataFrame as an attribute
                    self.post_exceptions = schema_exceptions_df.copy()
                    del schema_exceptions_df, schema_exceptions
                    del num_cols, buffer, heads
                    del indexer, index_uniques, index
            
            # return empty DataFrames if no exceptions are found
            if not pre_check:
                self.pre_exceptions = pd.DataFrame() # no PRE exceptions found
            if not post_check:
                self.post_exceptions = pd.DataFrame() # no POST exceptions found
        
        # raise .csv schema error where exceptions are found
        if pre_check and post_check:
            raise CSVSchemaError((f'.csv table schemas do not match for both PRE and POST files.'))
        elif pre_check:
            raise CSVSchemaError((f'.csv table schemas do not match for the PRE files.'))
        elif post_check:
            raise CSVSchemaError((f'.csv table schemas do not match for the POST files.'))
            
        del csvs_set, pre_check, post_check, check_sum

In [36]:
SetUp = DataSetUp()

In [37]:
SetUp.check_schemas()

CSVSchemaError: .csv table schemas do not match for both PRE and POST files.

In [38]:
SetUp.pre_exceptions

,AW19 W50 PRE.csv,AW18 W50 PRE.csv,SS19 W22 PRE.csv,SS21 W20 PRE.csv,SS20 W20 PRE.csv,AW20 W48 PRE.csv,SS18 W22 PRE.csv
19,CA Net TTC\r\nSem.\r\nN,CA Net TTC\nSem.\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\nSem.\nN
20,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\nSem.-1\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\nSem.-1\nN
21,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\nSem.-2\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\nSem.-2\nN
22,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\nSem.-3\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\nSem.-3\nN
23,Qté vendues\r\nSem.\r\nN,Qté vendues\nSem.\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\nSem.\nN
24,Qté vendues\r\nSem.-1\r\nN,Qté vendues\nSem.-1\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\nSem.-1\nN
25,Qté vendues\r\nSem.-2\r\nN,Qté vendues\nSem.-2\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\nSem.-2\nN
26,Qté vendues\r\nSem.-3\r\nN,Qté vendues\nSem.-3\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\nSem.-3\nN
27,Taux de remise\r\nSem.\r\nN,Taux de remise\nSem.\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\nSem.\nN
28,Taux de remise\r\nSem.-1\r\nN,Taux de remise\nSem.-1\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\nSem.-1\nN


In [39]:
SetUp.post_exceptions

,AW20 W2 POST.csv,AW18 W3 POST.csv,SS19 W27 POST.csv,SS21 W27 POST.csv,SS18 W27 POST.csv,AW19 W3 POST.csv,SS20 W27 POST.csv
19,CA Net TTC\r\nSem.\r\nN,CA Net TTC\nSem.\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\nSem.\nN,CA Net TTC\r\nSem.\r\nN,CA Net TTC\r\nSem.\r\nN
20,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\nSem.-1\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\nSem.-1\nN,CA Net TTC\r\nSem.-1\r\nN,CA Net TTC\r\nSem.-1\r\nN
21,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\nSem.-2\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\nSem.-2\nN,CA Net TTC\r\nSem.-2\r\nN,CA Net TTC\r\nSem.-2\r\nN
22,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\nSem.-3\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\nSem.-3\nN,CA Net TTC\r\nSem.-3\r\nN,CA Net TTC\r\nSem.-3\r\nN
23,Qté vendues\r\nSem.\r\nN,Qté vendues\nSem.\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\nSem.\nN,Qté vendues\r\nSem.\r\nN,Qté vendues\r\nSem.\r\nN
24,Qté vendues\r\nSem.-1\r\nN,Qté vendues\nSem.-1\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\nSem.-1\nN,Qté vendues\r\nSem.-1\r\nN,Qté vendues\r\nSem.-1\r\nN
25,Qté vendues\r\nSem.-2\r\nN,Qté vendues\nSem.-2\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\nSem.-2\nN,Qté vendues\r\nSem.-2\r\nN,Qté vendues\r\nSem.-2\r\nN
26,Qté vendues\r\nSem.-3\r\nN,Qté vendues\nSem.-3\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\nSem.-3\nN,Qté vendues\r\nSem.-3\r\nN,Qté vendues\r\nSem.-3\r\nN
27,Taux de remise\r\nSem.\r\nN,Taux de remise\nSem.\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\nSem.\nN,Taux de remise\r\nSem.\r\nN,Taux de remise\r\nSem.\r\nN
28,Taux de remise\r\nSem.-1\r\nN,Taux de remise\nSem.-1\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\nSem.-1\nN,Taux de remise\r\nSem.-1\r\nN,Taux de remise\r\nSem.-1\r\nN
